# Importando as bibliotecas que vão ser usadas 

In [1]:
import pandas as pd
import re
import matplotlib.pyplot as plt
import plotly.offline as py
import plotly.graph_objs as go
import plotly.express as px
import seaborn as sns

## Carregando a base de dados da pesquisa 

In [2]:
df = pd.read_csv("df_filmes_populares.csv")
df.drop(columns = "Unnamed: 0", inplace = True)

In [3]:
df

,ranking,titulo,ano_filme,duracao,genero_primario,generos_secundarios,generos,nota,descricao,diretor,estrelas,votos,faturamento,genero_link
0,1,Shang-Chi e a Lenda dos Dez Anéis,2021.0,132.0,Action,"[' Adventure', ' Fantasy']","['Action', ' Adventure', ' Fantasy']",8.0,"Shang-Chi, the master of unarmed weaponry-base...",['Destin Daniel Cretton'],"['Simu Liu', ' Awkwafina', ' Tony Chiu-Wai Leu...",67200.0,NaN,action
1,2,Matrix Resurrections,2021.0,NaN,Action,[' Sci-Fi'],"['Action', ' Sci-Fi']",NaN,The plot is currently unknown.,['Lana Wachowski'],"['Keanu Reeves', ' Jessica Henwick', ' Yahya A...",NaN,NaN,action
2,3,Maligno,2021.0,111.0,Action,"[' Crime', ' Horror']","['Action', ' Crime', ' Horror']",6.3,Madison is paralyzed by shocking visions of gr...,['James Wan'],"['Annabelle Wallis', ' Maddie Hasson', ' Georg...",20362.0,NaN,action
3,4,Free Guy: Assumindo o Controle,2021.0,115.0,Action,"[' Adventure', ' Comedy']","['Action', ' Adventure', ' Comedy']",7.6,A bank teller discovers that he's actually an ...,['Shawn Levy'],"['Ryan Reynolds', ' Jodie Comer', ' Taika Wait...",53199.0,NaN,action
4,5,O Esquadrão Suicida,2021.0,132.0,Action,"[' Adventure', ' Comedy']","['Action', ' Adventure', ' Comedy']",7.4,"Supervillains Harley Quinn, Bloodsport, Peacem...",['James Gunn'],"['Margot Robbie', ' Idris Elba', ' John Cena',...",195527.0,NaN,action
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
194158,"8,832",Cowboy,NaN,NaN,Western,[],['Western'],NaN,Add a Plot,[''],[''],NaN,NaN,western
194159,"8,833",O Agente Desconhecido,1936.0,57.0,Music,"[' Romance', ' Western']","['Music', ' Romance', ' Western']",4.8,Bob Allen in his starring debut gets a job on ...,['Spencer Gordon Bennet'],"['Robert Allen', ' Martha Tibbetts', ' Harry W...",24.0,NaN,western
194160,"8,834",Romance do Sertão,1919.0,50.0,Drama,[' Western'],"['Drama', ' Western']",NaN,Dreamer dairyman Phineas Dobbs of Cow Hollow s...,['Arthur Rosson'],"['Tom Mix', ' Juanita Hansen', ' Pat Chrisman'...",NaN,NaN,western
194161,"8,835",Trumpin' Trouble,1926.0,NaN,Western,[],['Western'],NaN,"John Lawson stakes out a gold mining claim, bu...",['Richard Thorpe'],"['Jay Wilsey', ' Bob Fleming', ' Alma Rayford'...",NaN,NaN,western


## Início do processo de manipulação das bases e montar os rankings 

In [4]:
# Calculando a nota média de cada gênero
media_por_genero_link = df.groupby(by = df["genero_link"])[["nota"]].mean()

In [5]:
# Calculando o faturamento e a quantidade de votos em cada gênero
fat_votos_genero_link = df.groupby(by = df["genero_link"])[["faturamento", "votos"]].sum()

In [6]:
# Mesclando as 2 tabelas anteriores
df_metricas_genero_link = media_por_genero_link.join(fat_votos_genero_link)

In [8]:
len(df_metricas_genero_link)

22

In [9]:
# Montando a lista que vai ser o ranking, como temos 22 gêneros, temos que separar 22 posições
lista_nota = list()
for nota in range(1, 23):
    lista_nota.append(nota)

## O dataframe é ordenado pelo critério em questão e depois é rankeado de 1 a 22 

In [10]:
df_metricas_genero_link.sort_values(by = "nota", ascending = False, inplace=True)

In [11]:
df_metricas_genero_link["ranking_nota"] = lista_nota

In [12]:
df_metricas_genero_link.sort_values(by = "faturamento", ascending = False, inplace=True)

In [13]:
df_metricas_genero_link["ranking_faturamento"] = lista_nota

In [14]:
df_metricas_genero_link.sort_values(by = "votos", ascending = False, inplace=True)

In [15]:
df_metricas_genero_link["ranking_votos"] = lista_nota

## A soma das 3 notas vai definir a classificação geral:
* Ranking de nota
* Ranking de votos
* Ranking de faturamento
> A soma dos 3 resulta no ranking geral
* Quanto menor a nota, mais no topo da tabela fica:
> Quanto menor a nota melhor

In [16]:
df_metricas_genero_link["ranking_geral"] = df_metricas_genero_link["ranking_faturamento"] + df_metricas_genero_link["ranking_nota"] + df_metricas_genero_link["ranking_votos"]

In [17]:
df_metricas_genero_link.sort_values(by = "ranking_geral", ascending = True, inplace=True)

## Levando em consideração nota, votos e faturamento esse é o nosso top 3 melhores gêneros: 

In [20]:
df_metricas_genero_link[["ranking_geral", "nota", "votos", "faturamento"]]

,ranking_geral,nota,votos,faturamento
genero_link,,,,
superhero,7,6.418099,772501975.0,2.751624e+11
drama,7,6.583138,426009754.0,1.161367e+11
comedy,21,6.042087,261204434.0,1.249181e+11
adventure,26,5.858586,244311181.0,1.172006e+11
romance,27,6.276490,151518639.0,5.929133e+10
action,28,5.759287,293710790.0,1.102349e+11
thriller,29,5.841255,281179703.0,7.745033e+10
crime,31,6.081354,177016088.0,4.613599e+10
biography,31,6.576073,57496866.0,1.451413e+10


In [46]:
fig = px.bar(data_frame= df_metricas_genero_link, x = df_metricas_genero_link.index, y = df_metricas_genero_link["faturamento"])
fig.update_layout(title_text='Faturamento Por Gênero')
fig.show()

## Calculando o faturamento de cada gênero por ano 

In [121]:
df_fat_por_ano_genero = pd.DataFrame()
df_fat_por_ano_genero

""


In [122]:
df_fat_por_ano_genero = df.groupby(by = ["genero_link", "ano_filme"]).sum()

duracao   nota    votos  faturamento
genero_link ano_filme                                      
action      1906.0        70.0    6.1    694.0          0.0
            1914.0       419.0   12.8   1110.0          0.0
            1915.0       421.0    7.3   4562.0          0.0
            1916.0       105.0    6.2   1675.0    8000000.0
            1918.0        73.0    5.8    782.0    3270000.0
...                        ...    ...      ...          ...
western     2020.0      3814.0  180.4  91183.0          0.0
            2021.0      4625.0  147.7   8156.0          0.0
            2022.0       291.0    0.0      0.0          0.0
            2023.0         0.0    0.0      0.0          0.0
            2024.0         0.0    0.0      0.0          0.0

[2370 rows x 4 columns]

In [134]:
lista_ano = list()
for ano in df_fat_por_ano_genero.index:
    lista_ano.append(ano[1])

lista_genero = list()
for ano in df_fat_por_ano_genero.index:
    lista_genero.append(ano[0])

In [136]:
df_fat_por_ano_genero["ano_adj"] = lista_ano
df_fat_por_ano_genero["genero_adj"] = lista_genero

In [153]:
fig = px.line(df_fat_por_ano_genero, x = "ano_adj", y = "faturamento", color = "genero_adj")
fig.update_layout(title_text='Faturamento Por Gênero e Ano')
fig.show()